<a href="https://colab.research.google.com/github/Daivar/Deep_Learning_Models/blob/main/Bike_sharing_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.kaggle.com/marklvl/bike-sharing-dataset
# !wget https://www.kaggle.com/marklvl/bike-sharing-dataset/download -O data.zip # need to add auth data
!wget https://github.com/MindaugasBernatavicius/DeepLearningCourse/raw/master/06_Regression_with_Neural_Networks_and_Tabular_Data/28865_36778_bundle_archive.zip 

!unzip -n *archive.zip
# !rm -rf bike-sharing-dataset/
!rm -f *.txt *.csv
!rm -f my_model

In [ ]:
import pandas as pd
import torch
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch.utils.data as data_utils 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

data = pd.read_csv('./bike-sharing-dataset/hour.csv', index_col=0)
data["season"].replace({1: "spring", 2: "summer", 3: "fall", 4: "winter"}, inplace=True)
data.head()

data = pd.get_dummies(data, columns= ['season'])
data.sample(5)

In [ ]:
# data prep
columns = ['registered', 'holiday', 'weekday', 
           'weathersit', 'temp', 'atemp',
           'season_fall', 'season_spring', 
           'season_summer', 'season_winter']

features = data[columns]
target = data[['cnt']]

X_train, x_test, Y_train, y_test = train_test_split(features, target, test_size=0.2)

X_train_tensor = torch.tensor(X_train.values, dtype = torch.float)
x_test_tensor = torch.tensor(x_test.values, dtype = torch.float)

Y_train_tensor = torch.tensor(Y_train.values, dtype = torch.float)
y_test_tensor = torch.tensor(y_test.values, dtype = torch.float)



# model
inp = X_train_tensor.shape[1]
out = 1
hid = 10
loss_fn = torch.nn.MSELoss()
batch_size = 10
shuffle = True

train_data = data_utils.TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = data_utils.DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)


# model = torch.nn.Sequential(torch.nn.Linear(inp, hid),
#                             torch.nn.Linear(hid, out))

model = torch.nn.Sequential(torch.nn.Linear(inp, hid),
                            torch.nn.ReLU(),
                            torch.nn.Linear(hid, out))

# model = torch.nn.Sequential(torch.nn.Linear(inp, hid),
#                             torch.nn.ReLU(),
#                             torch.nn.Dropout(p=0.2),
#                             torch.nn.Linear(hid, out))

# model = torch.nn.Sequential(torch.nn.Linear(inp, hid),
#                             torch.nn.Sigmoid(),
#                             torch.nn.Linear(hid, out))

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
total_step = len(train_loader)

# train loop
num_epochs = 10
for epoch in range(num_epochs + 1):
    for i, (features, target) in enumerate(train_loader):
        output = model(features)        # get predictions of the current model
        loss = loss_fn(output, target)  # calculate the loss: y vs. y_hat 
        optimizer.zero_grad()           # nulify previous gradients
        loss.backward()                 # perform backprop
        optimizer.step()                # perform weight updates
        
        if epoch % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
model.eval()
with torch.no_grad():
    y_pred_tensor = model(x_test_tensor)

y_pred = y_pred_tensor.detach().numpy()
compare_df = pd.DataFrame({'actual': np.squeeze(y_test.values), 'predicted': np.squeeze(y_pred)})
compare_df.sample(20)

print(sklearn.metrics.r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))